In [9]:
from google.colab import files

uploaded = files.upload()

Saving Production_Crops_Livestock_E_All_Data.csv to Production_Crops_Livestock_E_All_Data.csv


In [10]:
import pandas as pd

file_path = 'Production_Crops_Livestock_E_All_Data.csv'

df = pd.read_csv(file_path)

<ipython-input-10-d4c20b342025>:5: DtypeWarning: Columns (11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,65,68,71,74,77,80,83,86,89,92,95,98,101,104,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158,161,164,167,170,173,176,179,182,185,188,191,194,197) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [11]:
# Step 1: Keep base info columns
base_cols = ['Area', 'Item', 'Element', 'Unit']

# Step 2: Keep only year columns from Y2000 to Y2023 (exclude F and N)
year_cols = [col for col in df.columns if col.startswith('Y')
             and not col.endswith(('F', 'N'))
             and int(col[1:]) >= 2000]

# Step 3: Subset the dataframe
df_trimmed = df[base_cols + year_cols]

# Step 4: Preview the cleaned structure
df_trimmed.head()

,Area,Item,Element,Unit,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,...,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,Y2022,Y2023
0,Afghanistan,"Almonds, in shell",Area harvested,ha,7000.0,9000.0,5500.00,5700.0,12000.0,11768.0,...,13703.0,14676.0,19481.0,19793.0,20053.0,29203.0,22134.0,36862.0,36462.0,37000.0
1,Afghanistan,"Almonds, in shell",Yield,kg/ha,1714.3,1666.7,2140.70,2456.1,1225.0,1328.1,...,1999.6,1652.1,1685.9,1378.8,1716.1,1308.3,1775.9,1743.2,1742.0,1810.8
2,Afghanistan,"Almonds, in shell",Production,t,12000.0,15000.0,11773.99,14000.0,14700.0,15630.0,...,27400.0,24246.0,32843.0,27291.0,34413.0,38205.0,39307.0,64256.0,63515.0,67000.0
3,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",Area harvested,ha,4000.0,1600.0,3300.00,6800.0,15000.0,16000.0,...,30000.0,25000.0,24500.0,26160.0,25220.0,25293.0,25558.0,25357.0,25403.0,25439.0
4,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",Yield,kg/ha,625.0,625.0,606.10,602.9,600.0,625.0,...,716.7,720.0,707.5,697.0,708.0,704.1,703.0,705.0,704.0,704.0


In [13]:
# Step 1: Extract only the value columns (Y2000–Y2023)
value_cols = [col for col in df_trimmed.columns if col.startswith('Y') and col[1:5].isdigit()]

# Step 2: Melt the value columns into long format
df_long = df_trimmed.melt(
    id_vars=['Area', 'Item', 'Element', 'Unit'],
    value_vars=value_cols,
    var_name='Year',
    value_name='Value'
)

# Step 3: Standardize 'Year' column (convert 'Y2000' → 2000)
df_long['Year'] = df_long['Year'].str.extract(r'Y(\d+)').astype(int)

# Step 4: Reorder columns (no more Flag)
df_long = df_long[['Area', 'Item', 'Element', 'Year', 'Unit', 'Value']]

# Step 5: Preview result
df_long.head()

,Area,Item,Element,Year,Unit,Value
0,Afghanistan,"Almonds, in shell",Area harvested,2000,ha,7000.0
1,Afghanistan,"Almonds, in shell",Yield,2000,kg/ha,1714.3
2,Afghanistan,"Almonds, in shell",Production,2000,t,12000.0
3,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",Area harvested,2000,ha,4000.0
4,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",Yield,2000,kg/ha,625.0


In [14]:
# Count missing (NaN) values per column
missing_counts = df_long.isna().sum()
print("🔍 Missing values per column:")
print(missing_counts)

🔍 Missing values per column:
Area            0
Item            0
Element         0
Year            0
Unit            0
Value      219722
dtype: int64


In [15]:
# Count how many rows are missing the actual data value
missing_value_count = df_long['Value'].isna().sum()
print(f"\n❗ Missing 'Value' entries: {missing_value_count} out of {len(df_long)}")


❗ Missing 'Value' entries: 219722 out of 1876128


In [17]:
# Sort by Area, Item, Element, and then Year (chronological order)
df_long = df_long.sort_values(by=['Area', 'Item', 'Element', 'Year']).reset_index(drop=True)

# Preview to confirm
df_long.head(10)

,Area,Item,Element,Year,Unit,Value
0,Afghanistan,"Almonds, in shell",Area harvested,2000,ha,7000.0
1,Afghanistan,"Almonds, in shell",Area harvested,2001,ha,9000.0
2,Afghanistan,"Almonds, in shell",Area harvested,2002,ha,5500.0
3,Afghanistan,"Almonds, in shell",Area harvested,2003,ha,5700.0
4,Afghanistan,"Almonds, in shell",Area harvested,2004,ha,12000.0
5,Afghanistan,"Almonds, in shell",Area harvested,2005,ha,11768.0
6,Afghanistan,"Almonds, in shell",Area harvested,2006,ha,12000.0
7,Afghanistan,"Almonds, in shell",Area harvested,2007,ha,12000.0
8,Afghanistan,"Almonds, in shell",Area harvested,2008,ha,12000.0
9,Afghanistan,"Almonds, in shell",Area harvested,2009,ha,11029.0


In [18]:
# Drop rows where 'Value' is NaN (missing)
df_long = df_long[df_long['Value'].notna()].reset_index(drop=True)

In [19]:
# Drop rows where 'Value' is NaN (missing)
df_long = df_long[df_long['Value'].notna()].reset_index(drop=True)

# Confirm the result
print(f"✅ Remaining rows after dropping missing values: {len(df_long)}")

✅ Remaining rows after dropping missing values: 1656406


In [21]:
# Keep only rows where the Element is 'Production', as we are focusing on total output;
# this removes Area harvested, Yield, Stocks, and other elements
df_long = df_long[df_long['Element'] == 'Production'].reset_index(drop=True)

In [23]:
# Count how many unique items are in the dataset
unique_items = df_long['Unit'].nunique()

# List all unique items (optional)
item_list = df_long['Unit'].unique()

print(f"🔢 Total unique items: {unique_items}")
print("🧾 List of unique items:")
print(item_list)


🔢 Total unique items: 2
🧾 List of unique items:
['t' '1000 No']


In [27]:
# Keep only rows where Unit is NOT '1000 No'
df_long = df_long[df_long['Unit'] != '1000 No'].reset_index(drop=True)

# Confirm result
print("✅ Remaining rows after dropping '1000 No' unit:", len(df_long))
print("📌 Unique units now:", df_long['Unit'].unique())

✅ Remaining rows after dropping '1000 No' unit: 637986
📌 Unique units now: ['t']


In [28]:
df_long.head(5)

,Area,Item,Element,Year,Unit,Value
0,Afghanistan,"Almonds, in shell",Production,2000,t,12000.00
1,Afghanistan,"Almonds, in shell",Production,2001,t,15000.00
2,Afghanistan,"Almonds, in shell",Production,2002,t,11773.99
3,Afghanistan,"Almonds, in shell",Production,2003,t,14000.00
4,Afghanistan,"Almonds, in shell",Production,2004,t,14700.00


In [29]:
# Group by Area and Year, and sum total production
df_total_prod = df_long.groupby(['Area', 'Year'], as_index=False)['Value'].sum()

# Rename the 'Value' column for clarity
df_total_prod.rename(columns={'Value': 'Total_Production_tonnes'}, inplace=True)

# Preview
df_total_prod.head()

,Area,Year,Total_Production_tonnes
0,Afghanistan,2000,13811527.19
1,Afghanistan,2001,12736638.55
2,Afghanistan,2002,17921405.78
3,Afghanistan,2003,19173732.82
4,Afghanistan,2004,17323351.27


In [33]:
# Pivot to create one row per country, with years as columns
df_pivot = df_total_prod.pivot(index='Area', columns='Year', values='Total_Production_tonnes')

# Optional: Fill missing values with 0 (if any years are missing)
df_pivot = df_pivot.fillna(0)

# Reset index to keep 'Area' as a column
df_pivot = df_pivot.reset_index()

pd.options.display.float_format = '{:,.0f}'.format

# Preview the compressed dataset
df_pivot.head()

Year,Area,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,"13,811,527","12,736,639","17,921,406","19,173,733","17,323,351","21,530,410","19,772,050","22,458,549","18,610,014",...,"27,572,888","25,257,854","27,220,806","25,148,222","23,062,271","27,993,817","29,862,680","28,612,569","28,084,484","28,718,466"
1,Africa,"1,181,513,910","1,215,353,202","1,264,937,143","1,327,707,358","1,368,727,472","1,428,876,229","1,485,758,384","1,457,255,097","1,518,234,510",...,"1,908,533,924","1,925,985,353","1,947,489,814","2,033,771,853","2,126,038,943","2,133,518,940","2,205,511,446","2,250,538,042","2,294,207,898","2,252,379,239"
2,Albania,"5,435,581","5,519,840","5,613,244","5,716,665","5,855,444","5,972,510","6,152,443","6,023,784","6,471,557",...,"7,973,569","8,224,301","8,546,697","8,643,266","8,612,554","8,652,024","8,652,502","8,680,574","8,741,522","8,245,940"
3,Algeria,"17,650,570","21,618,827","21,566,679","28,993,511","30,720,471","31,304,504","33,231,687","30,426,134","28,899,827",...,"52,436,346","54,944,522","55,553,918","55,864,523","62,072,703","63,999,458","62,741,106","56,867,841","61,164,631","62,657,889"
4,Americas,"4,010,754,908","4,029,719,182","4,037,960,081","4,299,056,039","4,509,881,315","4,466,936,581","4,576,666,248","4,946,534,378","5,192,856,764",...,"5,856,125,060","5,939,113,138","6,133,386,673","6,111,189,830","6,031,351,082","6,077,698,641","6,182,815,573","6,089,088,824","6,072,003,068","6,017,158,011"


In [34]:
df_pivot.to_csv("Total_Production_Pivot.csv", index=False)

from google.colab import files
files.download("Total_Production_Pivot.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>